In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re 

In [2]:
def delete_word(sentence, word): 
    start = sentence.find(word) 
    if start >= 0: 
        return sentence[0:start] + sentence[start+len(word):]
    else: 
        return sentence 

In [3]:
data = pd.read_csv('standarts.csv', sep = ';', header = None, names = ['standarts', 'keys', 'name', 'year', 'title', 'releaseDate', 'price', 'abstract'])
data['year'] = np.zeros(len(data))
data

,standarts,keys,name,year,title,releaseDate,price,abstract
0,IEC 60880,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,IEC 61513,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,IEC 61508,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,IEC 60671,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,IEC 60964,NaN,NaN,0.0,NaN,NaN,NaN,NaN
5,IEC 60987,NaN,NaN,0.0,NaN,NaN,NaN,NaN
6,IEC 62340,NaN,NaN,0.0,NaN,NaN,NaN,NaN
7,IEC 61000-4-1,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [16]:
n = len(data)
for i in range(n):
    print(i)
    name_number = data['standarts'][i].split(' ')[1]
    url = "https://webstore.iec.ch/searchkey&RefNbr=&tag=&key="+name_number+"&RefNbr=&RefHeader=&ComNumber=&ICSNumber=&typepub=&vap=&From=&To=&start=1&MAX=all&FUZZY=0?_=1649102300560"
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    
    publications = soup.find('body').find('ul', class_="search-results lined-list").find_all('a')
    
    for publication in publications:
        
        s = str(publication)
        if(s.find(name_number) == -1): continue

        key = ''
        k = 0
        start = s.find('publication/')
        while('0' <= s[start+len('publication/') + k] and s[start+len('publication/') + k] <= '9'): 
            key += s[start+len('publication/') + k]
            k = k + 1
        
        name = s[s.find('IEC'):s.find('IEC')+3+1+len(name_number)+1+4]
        data.loc[i,'name'] = name
        
        match_20 = re.search("20\d\d", str(name))
        year_20 = (int(match_20[0]) if match_20 else 0)
    
        match_19 = re.search("19\d\d", str(name))
        year_19 = (int(match_19[0]) if match_19 else 0)
        
        if(max(year_20, year_19) >= data['year'][i]):
            data.loc[i,'keys'] = key
            data.loc[i,'year'] = max(year_20, year_19)
            data.loc[i,'name'] = name

0


AttributeError: 'NoneType' object has no attribute 'find'

In [5]:
data

,standarts,keys,name,year,title,releaseDate,price,abstract
0,IEC 60880,3795,IEC 60880:2006,2006.0,NaN,NaN,NaN,NaN
1,IEC 61513,5532,IEC 61513:2011,2011.0,NaN,NaN,NaN,NaN
2,IEC 61508,22273,IEC 61508</a>,2010.0,NaN,NaN,NaN,NaN
3,IEC 60671,2837,IEC 60671:2007,2007.0,NaN,NaN,NaN,NaN
4,IEC 60964,33054,IEC 60964:2018,2018.0,NaN,NaN,NaN,NaN
5,IEC 60987,64120,IEC 60987:2021,2021.0,NaN,NaN,NaN,NaN
6,IEC 62340,6874,IEC 62340:2007,2007.0,NaN,NaN,NaN,NaN
7,IEC 61000-4-1,24660,IEC TR 61000-4-1:2,0.0,NaN,NaN,NaN,NaN


In [6]:
n = len(data)
for i in range(n):
    name_number = data['standarts'][i].split(' ')[1]
    url = 'https://webstore.iec.ch/publication/'+data['keys'][i]
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    
    data.loc[i,'title'] = delete_word(delete_word(str(soup.find('title')), "<title>"), "</title>")
    data.loc[i,'title'] = data['title'][i].split('|')[0] 
    
    data.loc[i,'abstract'] = str(soup.find('div', class_ = 'abstract'))
    start = data['abstract'][i].find('description')
    data.loc[i,'abstract'] = data['abstract'][i][start+len('description')+2:]
    
    data.loc[i,'releaseDate'] = str(soup.find(itemprop = 'releaseDate'))

In [7]:
reg_date_tech = lambda s: re.search("\d{4}-\d\d-\d\d", s) 
reg_date = lambda s: reg_date_tech(s)[0] if reg_date_tech(s) else s 
 
data["releaseDate"] = data["releaseDate"].apply(reg_date)

In [8]:
data

,standarts,keys,name,year,title,releaseDate,price,abstract
0,IEC 60880,3795,IEC 60880:2006,2006.0,IEC 60880:2006,2006-05-09,NaN,Provides requirements for the software of comp...
1,IEC 61513,5532,IEC 61513:2011,2011.0,IEC 61513:2011,2011-08-25,NaN,Instrumentation and control (I&amp;C) systems ...
2,IEC 61508,22273,IEC 61508</a>,2010.0,IEC 61508:2010 CMV,2010-04-30,NaN,IEC 61508:2010 CMV contains the 2010 revision ...
3,IEC 60671,2837,IEC 60671:2007,2007.0,IEC 60671:2007,2007-05-23,NaN,Lays down principles for testing I&amp;C syste...
4,IEC 60964,33054,IEC 60964:2018,2018.0,IEC 60964:2018,2018-11-16,NaN,<!-- NEW! -->IEC 60964:2018 is available as <a...
5,IEC 60987,64120,IEC 60987:2021,2021.0,IEC 60987:2021,2021-02-03,NaN,IEC 60987:2021 provides requirements and recom...
6,IEC 62340,6874,IEC 62340:2007,2007.0,IEC 62340:2007,2007-12-07,NaN,Gives requirements related to the avoidance of...
7,IEC 61000-4-1,24660,IEC TR 61000-4-1:2,0.0,IEC TR 61000-4-1:2016,2016-04-27,NaN,IEC TR 61000-4-1:2016(E) gives information and...


In [9]:
data.to_csv('out.csv')

# Debug

In [10]:
i = 1

name_number = data['standarts'][i].split(' ')[1]
url = 'https://webstore.iec.ch/publication/'+data['keys'][i]
r = requests.get(url)
soup = bs(r.text, 'lxml')
    
soup

<!DOCTYPE html>
<html lang="en">
<head>
<script type="text/javascript">if(!navigator.cookieEnabled)window.location.href="http://webstore.iec.ch/webstore/webstore.nsf/xpPublication.xsp?Open\u0026pubid=5532\u0026test\u0026SessionID=G8MWL5B1N7";</script>
<title>IEC 61513:2011 | IEC Webstore | cyber security,  smart city</title>
<link href="/webstore_Build/favicon.ico" rel="SHORTCUT ICON"/>
<script djconfig="locale: 'en-us', parseOnLoad: true" src="/xsp/.ibmxspres/dojoroot-1.6.1/dojo/dojo.js" type="text/javascript"></script>
<script src="/xsp/.ibmxspres/dojoroot-1.6.1/ibm/xsp/widget/layout/layers/xspClientDojo.js" type="text/javascript"></script>
<script type="text/javascript">dojo.require('dojo.parser')</script>
<link href="/domjava/xsp/theme/webstandard/xsp.css" rel="stylesheet" type="text/css"/>
<link href="/domjava/xsp/theme/webstandard/xspLTR.css" rel="stylesheet" type="text/css"/>
<link href="/webstore_Build/css/iec-webstore.css" rel="stylesheet" type="text/css"/>
<script src="/webst

In [11]:
soup.find('div', class_ = 'tabbable')

<div class="tabbable buybox"><ul class="nav nav-tabs" id="buyboxb"><li class="active" id="view:_id75:_id78:repeat1:0:computedField2"><a data-toggle="tab" href=".paneE"> Download</a></li><li id="view:_id75:_id78:repeat1:1:computedField2"><a data-toggle="tab" href=".paneP"> Hardcopy</a></li></ul><div class="tab-content"><span class="xspTextComputedField" id="view:_id75:_id80:computedField1" style="color:rgb(255,0,0)"></span><div class="tab-pane fade active in paneE">
<span class="xspTextComputedField" id="view:_id75:_id80:repeat2:0:computedField2" style="color:rgb(255,0,0)"></span><form action="/webstore/webstore.nsf/xpPublication.xsp?Open&amp;pubid=5532&amp;test" class="E" enctype="multipart/form-data" id="view:_id75:_id80:repeat2:0:_id82" method="post">
<fieldset class="parameters"><div class="box-pane segment styled-select"><span class="xspTextComputedField" id="view:_id75:_id80:repeat2:0:_id82:computedField4" style="color:rgb(255,0,0)"></span><div class="langswitchlabel" id="view:_id

In [12]:
soup.find('div', class_ = 'tab-pane fade')

<div class="tab-pane fade" id="history"><span id="view:_id183:xmlHistory"></span></div>

In [13]:
soup.find('table', class_ = 'table table-striped')

<div class="box-pane segment"><span itemprop="offers" itemscope=" " itemtype="http://schema.org/Offer"><abbr itemprop="priceCurrency" title="Swiss Francs">CHF</abbr> <label class="price" content="345" id="view:_id75:_id80:repeat2:0:_id82:label1" itemprop="price">345.-<span class="xspTextComputedField" id="view:_id75:_id80:repeat2:0:_id82:computedField3"></span></label><meta content="http://schema.org/InStock" itemprop="availability"/></span></div>

In [14]:
soup.find('div', class_ = 'box-pane segment').find('label', class_='price').get('content')

'345'

In [15]:
str(soup)[start:start+5]

'D=G8M'